# Importação de bibliotecas
* Referências: https://theaisummer.com/cnn-architectures/#self-training-with-noisy-student-improves-imagenet-classification-2020
* Referências: https://www.linkedin.com/pulse/arquiteturas-de-redes-neurais-convolucionais-para-luiz-bianchi/
* Referências: https://lapix.ufsc.br/ensino/visao/visao-computacionaldeep-learning/deep-learningreconhecimento-de-imagens/

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Define o caminho do diretório base e o tamanho que as imagens vão ter

In [ ]:
img_width, img_height = 150, 150
# Diretório onde estão as imagens
train_data_dir = r'C:\Users\eluar\Documents\Estudo_IA_Python\Classificacao_Ronilson_projeto\Pequi'

# Função para carregar a base e dividir em conjunto de treino e teste e difinir as labels

In [ ]:
def carregaBase(caminho, target_size, batch_size=32, split_ratio=0.7):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=(1 - split_ratio))

    # Gerador de dados de treinamento
    train_generator = datagen.flow_from_directory(
        caminho,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        subset='training')  # Conjunto de treinamento
    
    # Gerador de dados de validação
    validation_generator = datagen.flow_from_directory(
        caminho,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale',
        subset='validation')  # Conjunto de validação
    
    return train_generator, validation_generator

# Carrega a base e mostra as labels 

In [ ]:
base_train, base_test = carregaBase(train_data_dir, target_size=(img_height, img_width))

#Imprime conjunto e labels 
class_indices = base_train.class_indices
print(class_indices)

# Função para avaliar os modelos

In [ ]:
def plotMatrizConfMD(matriz, modelo):
    #Plotar o grafico da matriz de confusão
    plt.figure(figsize=(8, 5))
    plt.title(f'Matriz de Confusao {modelo}')
    df_cm = pd.DataFrame(matriz, index=["DOENTE","SAUDAVEL"], columns=["DOENTE","SAUDAVEL"])
    ax = sns.heatmap(df_cm, cmap='Oranges', annot = True, fmt = '' )
    for text in ax.texts:
        text.set_text(f"{float(text.get_text())}")

    plt.show()

def avaliaModelo(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    
    # Calculando a especificidade
    conf_matrix = confusion_matrix(y_true, y_pred)
    specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    
    print(f'Acurácia: {accuracy}')
    print(f'Precisão: {precision}')
    print(f'Sensibilidade (Recall): {recall}')
    print(f'Especificidade: {specificity}')
    
    plotMatrizConfMD(conf_matrix, "MobileNet")

# Cria CNN AlexNet

In [ ]:
# Número de classes
num_classes = 2
def criaMobileNet(x_train, input_shape):
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='valid', activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # Compilando o modelo
    model.compile(loss='categorical_crossentropy',
                  optimizer= 'adam',
                  metrics=['accuracy'])
    # Treinando o modelo
    model.fit(x_train,
              #steps_per_epoch=len(train_generator),
              epochs=20)
    
    return model

# Executa o modelo AlexNet

In [ ]:
input_shape = (img_height, img_width, 1) 
MobileNet = criaMobileNet(base_train, input_shape)

# Obtendo os valores das imgens e suas labels 

In [ ]:
# Fazer previsões no conjunto de validação
x_test, y_test = zip(*(base_test[i] for i in range(len(base_test))))
x_test = np.concatenate(x_test)
y_test = np.concatenate(y_test)

# Realiza a Previsão

In [ ]:
# Prever as classes das imagens de validação
y_pred = MobileNet.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Avalia a Previsão 

In [ ]:
avaliaModelo(y_true_classes, y_pred_classes)